IMPORTING NECESSARY LIBRARIES FOR BUILDING MACHINE LEARNING MODEL AND CONVERTING WAV TO SPECTROGRAM

In [ ]:
PREPROCESS DATA AND SET DIRECTORIES

In [ ]:
parent_list = os.listdir(INPUT_DIRECTORY)
for i in range(3):
    print(parent_list[i])

CONVERSION OF WAVE TO SPECTROGRAM(IMAGES)

In [107]:
#creating new directory using os and converting files into spectrograms
for filename in os.listdir(INPUT_DIR):
    if "wav" in filename:
        file_path = os.path.join(INPUT_DIR, filename)
        file_stem = Path(file_path).stem
        target_dir = f'class_{file_stem[0]}'
        dist_dir = os.path.join(os.path.join(OUTPUT_DIR, 'audio-images'), target_dir)
        file_dist_path = os.path.join(dist_dir, file_stem)
        if not os.path.exists(file_dist_path + '.png'):
            if not os.path.exists(dist_dir):
                os.mkdir(dist_dir)
            file_stem = Path(file_path).stem
            sound_info, frame_rate = get_wav_info(file_path)
            pylab.specgram(sound_info, Fs=frame_rate)
            pylab.savefig(f'{file_dist_path}.png')
            pylab.close()

In [108]:
#CHECKING IF THE FILES ARE IN THE RIGHT PLACE (FOR TRAINING AND TESTING LATER)
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images'))
print("Classes: \n")
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images/healthy'))
print("\nA few example files: \n")

Classes: 


A few example files: 



In [109]:
#DECLARE CONSTANTS, THESE ARE THE ACCEPTABLE DIMENSIONS THAT THIS WILL TAKE (WE SHOULD CHANGE CLASSES TO OUR NEEDS AND BATCHES TO THE AMOUNT THAT WE WANT AS WELL)
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 32
N_CHANNELS = 3
N_CLASSES = 10

In [110]:
#MAKE A DATASET WITH THE TRAINING SPECTROGRAMS ESTABLISHED
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-images'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="training",
                                             seed=0)

Found 0 files belonging to 3 classes.
Using 0 files for training.


ValueError: No images found in directory OUTPUT\audio-images. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [ ]:
#CREATE DATASET WITH VALIDATION SPECTROGRAMS (IF MODEL MATCHES THIS IT IS GOOD)
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-images'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="validation",
                                             seed=0)

In [ ]:
#USE MATPLOTLIB TO VISUALISE (DO NOT USE THIS IN CODE)
plt.figure(figsize=(12, 12))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")
plt.show()

In [ ]:
#FORM A FUNCTION FOR THE MODEL USING A KERAS SEQUENTIAL MODEL
def prepare(ds, augment=False):
    rescale = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Rescaling(1./255)])
    flip_and_rotate = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
    ])

In [ ]:
#RESIZE THE MODEL TO FIT THE DEFINED CONSTANTS AT THE START OF THIS SECTION
ds = ds.map(lambda x, y: (rescale(x, training=True), y))
    if augment: ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y))
    return ds

train_dataset = prepare(train_dataset, augment=False)
valid_dataset = prepare(valid_dataset, augment=False)

In [ ]:
#CREATE DEEP LEARNING MODEL USING KERAS SEQUENTIAL CONV2D(CONVOLUTION) AND NORMALIZATION OF BATCH DATA AND FLATTEN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

In [ ]:
#USE KERAS OPTIMIZER TO COMPILE THE MODEL
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(),
    metrics=['accuracy'],
)

In [ ]:
#FINALLY TRAIN MODEL (MODIFY EPOCHS)
history = model.fit(train_dataset, epochs=500, validation_data=valid_dataset)

EVALUATING MODEL (NOT NECESSARY AS THE MODEL HAS ALREADY BEEN CREATED)

In [ ]:
# PLOT ACCURACY AND LOSS CURVES FOR VISUALISATION ONLY
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values)+1)

plt.figure(figsize=(8,6))
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Calculate final loss and accuracy values
final_loss, final_acc = model.evaluate(valid_dataset, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))